# Converting `.wav` dataset (train, test, split) to mel-spectrogram `.npy` dataset 

In [4]:
import os
import numpy as np
import librosa
from tqdm import tqdm

In [8]:
# Funkcja do generowania mel-spektrogramu i zapisu jako .npy
def generate_and_save_melspec(file_path, output_path):
    try:
        # Wczytanie pliku audio
        y, sr = librosa.load(file_path, sr=None)
        # Generowanie mel-spektrogramu
        melspec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
        melspec_db = librosa.power_to_db(melspec, ref=np.max)
        # Zapis do pliku .npy
        np.save(output_path, melspec_db)
        # print(f"Zapisano: {output_path}")
    except Exception as e:
        print(f"Błąd przetwarzania {file_path}: {e}")

# Przetwarzanie katalogów
def process_folders(source, target):
    for root, dirs, files in os.walk(source):
        # Tworzenie równoległej struktury katalogów w folderze docelowym
        relative_path = os.path.relpath(root, source)
        target_dir = os.path.join(target, relative_path)
        os.makedirs(target_dir, exist_ok=True)

        # Przetwarzanie plików .wav z paskiem postępu
        wav_files = [file for file in files if file.endswith('.wav')]
        for file in tqdm(wav_files, desc=f"Przetwarzanie folderu: {relative_path}", unit="plik"):
            source_file = os.path.join(root, file)
            target_file = os.path.join(target_dir, file.replace('.wav', '.npy'))
            result = generate_and_save_melspec(source_file, target_file)

In [9]:
source_path = '../../../datasets/jamendo/split_audio/'
target_path = '../../../datasets/jamendo/split_melspec/'

# Wywołanie funkcji
process_folders(source_path, target_path)

Przetwarzanie folderu: .: 0plik [00:00, ?plik/s]
Przetwarzanie folderu: val: 100%|██████████| 11482/11482 [21:34<00:00,  8.87plik/s]
